In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [3]:
df_data =pd.read_csv("DATA.CSV",index_col=0)
df_data.head(2).T

,0,1
Age,51,52
Attrition,No,No
BusinessTravel,NaN,NaN
DailyRate,"684,0$","699,0$"
Department,NaN,NaN
DistanceFromHome,6,1
Education,3,4
EducationField,NaN,Life Sciences
employeecount,1,1
employeenumber,"162,0","259,0"


In [5]:
df_data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'NUMBERCHILDREN', 'RemoteWork'],
      dtype='object')

##### Homogeneización del nombre de las columnas (en minúsculas)

In [11]:
nuevas_columnas={columna:columna.lower() for columna in df_data} #modificamos las columnas a minúsculas
df_data.rename(columns= nuevas_columnas, inplace= True) #cambiamos las columnas antiguas por las homogeneizadas en el paso anterior (nuevas_columnas)

In [12]:
df_data.head(2).T

,0,1
age,51,52
attrition,No,No
businesstravel,NaN,NaN
dailyrate,"684,0$","699,0$"
department,NaN,NaN
distancefromhome,6,1
education,3,4
educationfield,NaN,Life Sciences
employeecount,1,1
employeenumber,"162,0","259,0"


##### Limpieza y verificación de datos
- Modificar columnas categóricas a numéricas
- Estandarizar las etiquetas de las columnas categóricas (ej. gender, maritual status)
- Verificar duplicados
- Cambiar valores faltantes a nulos
- Dividir columnas y verificar su pertenencia (ej. "roledepartment" es una combinación de columnas "department" y "jobrole"?)
- Crear nuevas columnas (ej. de la columna "stock_option_level" crear unna columna con 2 categorías: "No" (stock_option_level = 0); "Yes" (stock_option_level = 1,2,3))
- Eliminar columnas redundantes y filas que sean duplicadas o en las que la mayoría de los valores de las columnas sean nulos